In [0]:
dbutils.library.installPyPI('h2o-pysparkling-2.4')
from pysparkling import * # Import PySparkling
import h2o
import pandas as pd
spark.conf.set("spark.sql.execution.arrow.enabled", "false")
from pyspark.sql.functions import *
import warnings
warnings.filterwarnings("ignore")
from pysparkling.ml import H2OAutoML
from pyspark.ml import Pipeline
from pyspark.ml.feature import SQLTransformer
from pyspark.sql.types import *
from datetime import datetime,date 

In [0]:
hc = H2OContext.getOrCreate()

Connecting to H2O server at http://10.139.64.4:9009 ... successful.
-------------------------- -------------------------------------------------------------------------------------------------------
H2O_cluster_uptime: 19 secs
H2O_cluster_timezone: Etc/UTC
H2O_data_parsing_timezone: UTC
H2O_cluster_version: 3.32.0.2
H2O_cluster_version_age: 15 days
H2O_cluster_name: sparkling-water-root_app-20201203023605-0000
H2O_cluster_total_nodes: 4
H2O_cluster_free_memory: 25.29 Gb
H2O_cluster_total_cores: 16
H2O_cluster_allowed_cores: 16
H2O_cluster_status: locked, healthy
H2O_connection_url: http://10.139.64.4:9009
H2O_connection_proxy: null
H2O_internal_security: False
H2O_API_Extensions: XGBoost, Algos, Amazon S3, Sparkling Water REST API Extensions, AutoML, Core V3, TargetEncoder, Core V4
Python_version: 3.7.3 final
-------------------------- -------------------------------------------------------------------------------------------------------

Sparkling Water Context:
 * Sparkling Water Version: 3.32.0.2-1-2.4
 * H2O name: root
 * cluster size: 4
 * list of used nodes:
 (executorId, host, port)
 ------------------------
 (0,10.139.64.5,54321)
 (1,10.139.64.6,54321)
 (2,10.139.64.7,54321)
 (3,10.139.64.8,54321)
 ------------------------

 Open H2O Flow in browser: https://westeurope.azuredatabricks.net/driver-proxy/o/8767374559588946/1016-113926-refer971/9009/flow/index.html (CMD + click in Mac OSX)

In [0]:
def read_data():
  '''Read all locations' train data'''
  all_data_df=spark.table('churn.train_data_2019_2020_all_loc')
  return all_data_df

def filter_data(all_data_df, location_id):
  '''Filter data by location id and prepare the dataframe for training'''

  train_df=all_data_df[all_data_df.location_id==location_id]
  train_df=train_df.drop('user_id','location_id','location_name','updated_on') # drop the columns not needed in train set
  train_df=train_df.withColumnRenamed('num_days_before_end2019_purchased','num_days_before_end_purchased') # renaming
  train_df=train_df.withColumn('tag_churn',train_df["tag_churn"].cast(StringType())) # type coversion
  return train_df 

def get_feature_importance(leader_board_pd):
  '''Get features importance of selected model bi model_id and plot them'''
  
  model_id=leader_board_pd["model_id"][2]
  m = h2o.get_model(model_id)
  
  print('Plot Feature Importance---')
  #print(m.varimp_plot()) #Variable importance
  
  print('Feature importance Values------')
  print(m.varimp(use_pandas=True))
  
  print(f'*** PLOTTING {model_id}***')
  #print(m.plot())
  
def get_metrics(leader_board_pd):
  '''compute the metrics of the model trained by id '''
  
  model_id=leader_board_pd["model_id"][0]
  model = h2o.get_model(model_id)
  
  print("Model Perfomance---")
  print(model.model_performance())
  print("--------------------")
  
  threshold=model.find_threshold_by_max_metric(metric='F1')
  precision=(model.precision(thresholds=threshold)[0][1])*100
  recall=(model.recall(thresholds=threshold)[0][1])*100
  accuracy=(model.accuracy(thresholds=threshold)[0][1])*100
  f1=model.F1(thresholds=threshold)[0][1]
  specificity=100*(model.tnr(thresholds=threshold)[0][1])
  mcc=model.mcc(thresholds=threshold)[0][1]
  
  print('Saving these metrics for leader model')
  print(f'Threshold to consider for mx f1 metric based is {threshold}')
  print(f'The model sensitivity/recall or total positive rate ie model will catch {recall} % of customers who will actually churn')
  print(f'The model specificity or True negative rate ie model will catch {specificity} % of customers who will actually Notchurn')
  print(f'Precision ie Out of all customers it predicted as it will churn {precision} % of them will actually churn')      
  print(f'Overall accuracy is {accuracy}%')
  print(f'Max F1 score {f1}')
  print(f'Mcc score is {mcc}')
  
  return threshold,precision,recall,accuracy,f1,specificity,mcc
  
def save_model(model, location_name, updated_on, best_model_df):
  '''save the trained leader model and metrics'''
  
  # SAVE MODEL
  location_name = location_name.replace(" ", "")
  print('Saving Model For:' , location_name)
  path_tomodel="/dbfs/FileStore/df/Churn_Models/h2o_leader_model_train_"+location_name+"_"+str(updated_on)+".model"
  model.write().overwrite().save(path_tomodel)
  
  # SAVE METRICS
 
  best_model_df_sp = spark.createDataFrame(best_model_df)
  best_model_df_sp.write.format('delta').mode('append').saveAsTable('churn.h2o_leader_model_train_metrics')
  
  
def train_model(train_df,location_id,location_name):
  '''Train the model, get the leaderboard and save them'''
  
  InputdataTransformer = SQLTransformer(statement="SELECT * FROM __THIS__")
  automlEstimator = H2OAutoML(maxModels=26,seed=1,splitRatio=0.75,nfolds=5,balanceClasses=True,labelCol="tag_churn")
  #Building Pipeline of Inputs and Estimator
  pipeline = Pipeline(stages=[InputdataTransformer, automlEstimator])
  
  # Train thr model 
  model = automlEstimator.fit(train_df)
  
  # Get leaderboard 
  lb = automlEstimator.getLeaderboard()
  return model, lb
  
def main_training_function():
  '''main function retrieving data for all locations, training the models and saving them'''
  
  # Get all the location ids and names
  inputs_df=spark.sql('select * from churn.all_location_inputs_train_test')
  all_locations_ids = [x["location_id"] for x in inputs_df.collect()]
  print('Total Locations=',len(all_locations_ids))
  
  # Get all locations train data
  all_data_df=read_data()
    
  # Train seprate models for each location in this loop
  for loc_id in all_locations_ids:
    if int(loc_id)>0:
      location_id = loc_id
      location_name = inputs_df.where(inputs_df.location_id == loc_id).select('location_name').collect()[0]['location_name']
      print(f'Start Training Model For Location ID: {loc_id} and Location Name: {location_name}')

      # Filter and prepare train data by location_id
      train_df=filter_data(all_data_df, location_id)

      # Model training and saving
      model, leader_board = train_model(train_df,location_id,location_name)

      leader_board_pd=leader_board.toPandas()

      get_feature_importance(leader_board_pd)
      threshold,precision,recall,accuracy,f1,specificity,mcc = get_metrics(leader_board_pd) 

      updated_on=date.today() 
      best_model_df=leader_board_pd[:1]
      best_model_df['location_name']=location_name
      best_model_df['updated_on']=updated_on
      best_model_df['threshold']=threshold
      best_model_df['precision']=precision
      best_model_df['recall']=recall
      best_model_df['accuracy']=accuracy
      best_model_df['f1']=f1
      best_model_df['specificity']=specificity
      best_model_df['mcc']=mcc

      save_model(model, location_name, updated_on,best_model_df)
    
  print('DONE....')
    
# def main_train_function_for_debugging():
#   '''Use this function to debug/test for one hardcoded location only'''  
#   # HARDCODE THE INPUTS HERE
#   location_id = 9
#   location_name = 'testrun_for_Qatar'
#   print(f'Start Training Model For Location ID: {location_id} and Location Name: {location_name}')

#   # Get all locations train data
#   all_data_df=read_data()

#   # Filter and prepare train data by location_id
#   train_df=filter_data(all_data_df, location_id)

#  # Model training and saving
#   model, leader_board = train_model(train_df,location_id,location_name)
  
#   try:
#     model_id=leader_board["model_id"][2]
#     print('model_id ',model_id)
#     feature_imp_model = get_feature_importance(model_id)

#     threshold,precision,recall,accuracy,f1,specificity,mcc = get_metrics(model) 
#     leader_board = leader_board.toPandas()
#     updated_on=date.today() 
#     best_model_df=leader_board[:1]
#     best_model_df['location_name']=location_name
#     best_model_df['updated_on']=updated_on
#     best_model_df['threshold']=threshold
#     best_model_df['precision']=precision
#     best_model_df['recall']=recall
#     best_model_df['accuracy']=accuracy
#     best_model_df['f1']=f1
#     best_model_df['specificity']=specificity
#     best_model_df['mcc']=mcc
#   except Exception as e:
#     print(e)
#   try:    
#     save_model(model, location_name, updated_on,best_model_df)
#   except Exception as e:
#     print('saving: ', e)
  
#   print('DONE....')
#   return model, leader_board#best_model_df, feature_imp_model


In [0]:
main_training_function()
# model, leader_board = main_train_function_for_debugging() # for testing/debugging purpose

Total Locations= 13
Start Training Model For Location ID: 1 and Location Name: Dubai
Plot Feature Importance---
Feature importance Values------
 variable ... percentage
0 diff_num_months_1st_last ... 0.244990
1 num_days_before_endyear_active_session ... 0.196151
2 num_of_months_redemptions ... 0.067098
3 num_days_before_end_purchased ... 0.064571
4 num_distinct_months_active ... 0.063033
5 num_days_before_endyear_active_redemption ... 0.039057
6 num_of_weeks_redemptions ... 0.035665
7 tag_nationality ... 0.034555
8 tenure_inapp ... 0.028962
9 total_spend ... 0.018037
10 avg_recency_days ... 0.015211
11 total_paid_for_product ... 0.015043
12 age_group ... 0.013491
13 cor_tag ... 0.012851
14 total_sessions_monthly ... 0.012454
15 home ... 0.009602
16 median_spend_redem ... 0.007731
17 total_owned_products.1 ... 0.007392
18 median_monthly_spend ... 0.006925
19 quicksearch ... 0.006080
20 median_num_days_permonth_active ... 0.005622
21 total_monthly_spend_redemptions ... 0.005599
22 num_of_merchants_interacted ... 0.005289
23 num_distinct_days_active ... 0.005238
24 avg_monthly_sessions_active ... 0.005126
25 total_owned_products.0 ... 0.004981
26 avg_recency_days_inapp ... 0.003992
27 count_num_areas ... 0.003909
28 merchantdetail ... 0.003654
29 total_sessions ... 0.003654
30 redemptioncard ... 0.003653
31 min_spend_redem ... 0.003586
32 offerslist ... 0.003549
33 count_of_total_redemptions ... 0.003445
34 percentage_activity_of_redeem_appvisit ... 0.003403
35 num_of_days_redemptions ... 0.003347
36 max_spend_redem ... 0.003100
37 notifications ... 0.003091
38 advancesearch ... 0.002984
39 categoryhome ... 0.002853
40 num_distinct_merchant_categories ... 0.002506
41 gender ... 0.002328
42 percentage_of_merchants_redeem_present ... 0.002143
43 total_owned_products.2 ... 0.002117
44 num_distinct_merchant_ids ... 0.002112
45 avg_weekly_sessions_active ... 0.001861
46 has_redem_buffet ... 0.001606
47 product ... 0.001590
48 offers ... 0.001512
49 favourites ... 0.001510
50 has_connect ... 0.000666
51 is_savings_morethan_paid ... 0.000422
52 offertime_of_purchase ... 0.000381
53 total_owned_products.3 ... 0.000230
54 zero_redeemer ... 0.000039
55 total_owned_products.4 ... 0.000000

[56 rows x 4 columns]
*** PLOTTING GBM_2_AutoML_20201203_092049***
Model Perfomance---

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.1304220385244704
RMSE: 0.36113991544063695
LogLoss: 0.4147123727066082
Null degrees of freedom: 9920
Residual degrees of freedom: 9899
Null deviance: 13668.939940787548
Residual deviance: 8228.72289924452
AIC: 8272.72289924452
AUC: 0.9023709184648403
AUCPR: 0.8957801615299349
Gini: 0.8047418369296806

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4408392098641903: 
 0 1 Error Rate
----- ---- ---- ------- ---------------
0 4533 885 0.1633 (885.0/5418.0)
1 885 3618 0.1965 (885.0/4503.0)
Total 5418 4503 0.1784 (1770.0/9921.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric threshold value idx
--------------------------- ----------- -------- -----
max f1 0.440839 0.803464 213
max f2 0.231759 0.863883 309
max f0point5 0.575577 0.832924 157
max accuracy 0.483107 0.823506 196
max precision 0.961692 1 0
max recall 0.108148 1 385
max specificity 0.961692 1 0
max absolute_mcc 0.483107 0.643292 196
max min_per_class_accuracy 0.421306 0.815012 221
max mean_per_class_accuracy 0.448098 0.820439 209
max tns 0.961692 5418 0
max fns 0.961692 4445 0
max fps 0.0693138 5418 399
max tps 0.108148 4503 385
max tnr 0.961692 1 0
max fnr 0.961692 0.98712 0
max fpr 0.0693138 1 399
max tpr 0.108148 1 385

Gains/Lift Table: Avg response rate: 45.39 %, avg score: 46.70 %
group cumulative_data_fraction lower_threshold lift cumulative_lift response_rate score cumulative_response_rate cumulative_score capture_rate cumulative_capture_rate gain cumulative_gain kolmogorov_smirnov
------- -------------------------- ----------------- --------- ----------------- --------------- -------- ----------

In [0]:
#predicted = model.transform(for_predictions)
# updated_on=date.today()

In [0]:
# %sql
# DROP TABLE IF EXISTS churn.h2o_leader_model_train_metrics